In [102]:
import os
import hardposit as hp
import softposit as sp
import fixed2float as fi
from pathlib import Path
import re
import ast

In [148]:
import fixed2float
fixed2float.__version__

'4.0.1'

In [149]:
N, ES = 16, 1

In [150]:
with open(Path("../tb_ppu.log"), 'r') as f:
  log = f.read()
print(log)

(0x3524, 0x5e81, 0x0000000284df2404, 0x5427)
(0xd609, 0x5663, 0x00000001623ce184, 0x4624)
(0x7b0d, 0x998d, 0xfffffa7ec75ce184, 0x81d0)
(0x8465, 0x5212, 0xfffff875902ce184, 0x818f)
(0xe301, 0xcd0d, 0xfffff875aef56324, 0x818f)
(0xf176, 0xcd3d, 0xfffff875b6a05504, 0x818f)
(0x57ed, 0xf78c, 0xfffff875a94ee884, 0x818f)
(0xe9f9, 0x24c6, 0xfffff875a033bd34, 0x818f)
(0x84c5, 0xd2aa, 0xfffff8d46505bd34, 0x819b)
(0xf7e5, 0x7277, 0xfffff8d3fadb8934, 0x819a)



In [151]:
log.splitlines()[0]

'(0x3524, 0x5e81, 0x0000000284df2404, 0x5427)'

In [152]:
lst = list(map(lambda l: ast.literal_eval(l), log.splitlines()))

In [153]:
operand1_i, operand2_i, fixed_o, result_o = list(zip(*lst)) 

In [154]:
operand1_i, operand2_i, fixed_o, result_o

((13604, 54793, 31501, 33893, 58113, 61814, 22509, 59897, 33989, 63461),
 (24193, 22115, 39309, 21010, 52493, 52541, 63372, 9414, 53930, 29303),
 (10819150852,
  5943124356,
  18446738021150417284,
  18446735782546563460,
  18446735783063020324,
  18446735783191663876,
  18446735782968223876,
  18446735782815448372,
  18446736189844471092,
  18446736188063320372),
 (21543, 17956, 33232, 33167, 33167, 33167, 33167, 33167, 33179, 33178))

In [155]:
acc = hp.from_bits(0, N, ES)
for (operand1, operand2) in zip(operand1_i, operand2_i):
  acc += (hp.from_bits(operand1, N, ES) * hp.from_bits(operand2, N, ES))

In [156]:
acc

1111111001100101 P<16,1>(33179) -1832.0

In [157]:
M, B = 31, 64
fi.from_bits(fixed_o[-1], M, B)

1111111111111111111110001101001111111010110110111000100100110100 Fx<31,64>(18446736188063320372) -1836.0200876472518

In [158]:
hp.from_bits(result_o[-1], N, ES).eval()

-1840.0

### test result using softposit

In [159]:
softposit_acc = [0] * len(operand1_i)

q = sp.quire16()

for i, (operand1, operand2) in enumerate(zip(operand1_i, operand2_i)):
  q.qma(
    sp.posit16(bits=operand1), sp.posit16(bits=operand2)
  )
  softposit_acc[i] = q.v

In [160]:
softposit_acc

[2.51904296875,
 1.3837890625,
 -1408.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1832.0,
 -1840.0]

### FPPU hardware outputs

In [161]:
results_o = list(map(lambda bits: hp.from_bits(bits, N, ES).eval(), result_o))
results_o

[2.51904296875,
 1.3837890625,
 -1408.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1832.0,
 -1840.0]

In [162]:
list(map(lambda val1, val2: val1 == val2, map(str, results_o), map(str, softposit_acc)))

[True, True, True, True, True, True, True, True, True, True]

In [163]:
assert all(_) == True
print("Congratulation, it works.")

Congratulation, it works.
